## Use-case demonstration of oocgcm grid objects

- **author** : Julien Le Sommer
- **date** : May 2016
- **purpose** : This notebook is used for testing/debugging oocgcm grid objects during the development process.

In [0]:
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

In [0]:
from oocgcm.oceanmodels.nemo import grids

In [0]:
#- Parameter
coordfile  = '/Users/lesommer/data/NATL60/NATL60-I/NATL60_coordinates_v4.nc'
maskfile = '/Users/lesommer/data/NATL60/NATL60-I/NATL60_v4.1_cdf_byte_mask.nc'
filenatl60 = '/Users/lesommer/data/NATL60/NATL60-MJM155-S/1d/2008/NATL60-MJM155_y2008m01.1d_BUOYANCYFLX.nc'

#chunks = (3454,5422)
chunks = (1727,2711)
xr_chunks = {'x': chunks[-1], 'y': chunks[-2]}
xr_chunks_t = {'x': chunks[-1], 'y': chunks[-2],'time_counter':1}

In [0]:
#- creating the grid object
grd = grids.nemo_2d_grid(nemo_coordinate_file=coordfile,nemo_byte_mask_file=maskfile,chunks=xr_chunks)

### with a 2D input xarray dataarray at a given time

In [0]:
#- defining a 2D xarray
sig0 = xr.open_dataset(filenatl60,chunks=xr_chunks,lock=False)['vosigma0'][20]

In [0]:
#- compute the horizontal gradient
g = grd.horizontal_gradient(sig0)

In [0]:
%time gx = g.x_component.load()

### with a 2D + time input xarray dataarray

In [0]:
#- defining a 2D xarray
sig0 = xr.open_dataset(filenatl60,chunks=xr_chunks_t,lock=False)['vosigma0']

In [0]:
#- compute the horizontal gradient
g = grd.horizontal_gradient(sig0)

In [0]:
%time gx = g.x_component[20].load()

### Compute a laplacian

In [0]:
lap = grd.horizontal_laplacian(sig0)
%time l = lap[20].load()

### plotting the output array

In [0]:
#- plot the horizontal gradient
plt.figure(figsize=(15,10))
l.plot.pcolormesh(vmin=-1e-15,vmax=1e-15)